In [1]:
import glob

from fastai.text import *
# from fastai.callbacks import SaveModelCallback
import sentencepiece as spm #https://github.com/google/sentencepiece

from prod.sp_tok import *

torch.backends.cudnn.enabled=False #for quant.313v machine.  Not sure why...

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

ModuleNotFoundError: No module named 'numpy.core._multiarray_umath'

In [2]:
PATH = Path('./data/rnn')
sp_model = './all_tweets_es_0606'

defaults.text_spec_tok.append(NL) #add a New Line special char
sp_vocab = Vocab( get_itos(sp_model) )
mycust_tok = CustomTokenizer(SPTokenizer,sp_model,pre_rules=default_rules)

In [3]:
test_df = pd.read_csv('./data/haha_2019_test.csv')
test_df['is_humor'] = 0
test_df['is_humor'].iloc[0] = 1 #create two classes in data.
## apply the rules
raw_text = test_df.loc[:,'text']

print("Default Rules:\n",[x.__name__ for x in default_rules],"\n\n")
for rule in default_rules:
    raw_text = raw_text.apply(lambda x: rule(str(x)))    
test_df['new_text'] =  raw_text #tokenizer adds xxbos

data_tst = TextClasDataBunch.from_df(PATH, test_df, test_df,
                               tokenizer=mycust_tok, vocab=sp_vocab,
                               text_cols='new_text', label_cols='is_humor')

In [4]:
model_wt_fnames = ['/haha_clas*.pth']
len(model_wt_fnames)

200

In [5]:
ensemble_preds = []
for wt_fname in model_wt_fnames:
    config = awd_lstm_clas_config.copy()
    config['qrnn'] = True
    config['n_hid'] = 2304
    
    learn = text_classifier_learner(data_tst, AWD_LSTM, drop_mult=0.7,pretrained=False,
                               config=config)
    learn = learn.load(wt_fname.split('.')[0]) #load weights
    preds_t,y_t,losses_t = learn.get_preds(DatasetType.Valid,with_loss=True, ordered=True)
    ensemble_preds.append(to_np(preds_t))

In [6]:
np.array(ensemble_preds).shape

(200, 6000, 2)

In [7]:
np.array(ensemble_preds)

array([[[0.318021, 0.681979],
        [0.862715, 0.137285],
        [0.065169, 0.934831],
        [0.916044, 0.083956],
        ...,
        [0.748863, 0.251137],
        [0.909519, 0.090481],
        [0.254185, 0.745815],
        [0.129196, 0.870804]],

       [[0.249954, 0.750046],
        [0.917265, 0.082735],
        [0.048775, 0.951225],
        [0.913822, 0.086178],
        ...,
        [0.873074, 0.126927],
        [0.901268, 0.098732],
        [0.248364, 0.751636],
        [0.170292, 0.829708]],

       [[0.20194 , 0.79806 ],
        [0.872554, 0.127446],
        [0.055917, 0.944083],
        [0.963631, 0.036369],
        ...,
        [0.951323, 0.048677],
        [0.935856, 0.064145],
        [0.546697, 0.453303],
        [0.144203, 0.855797]],

       [[0.194375, 0.805625],
        [0.931203, 0.068797],
        [0.082297, 0.917703],
        [0.99178 , 0.00822 ],
        ...,
        [0.978084, 0.021916],
        [0.963812, 0.036188],
        [0.434048, 0.565952],
        [0.1

In [9]:
ens_mean = np.array(ensemble_preds).mean(axis=0)
ens_mean

array([[0.215043, 0.784957],
       [0.902099, 0.097901],
       [0.057191, 0.942809],
       [0.954656, 0.045344],
       ...,
       [0.91606 , 0.08394 ],
       [0.930531, 0.069468],
       [0.427875, 0.572124],
       [0.171027, 0.828973]], dtype=float32)

In [10]:
out_df = test_df['id'].to_frame().copy()
out_df['is_humor'] = ens_mean.argmax(axis=1)
out_df['funniness_average'] = 3.

In [11]:
out_df.head()

,id,is_humor,funniness_average
0,tweet1,1,3.0
1,tweet2,0,3.0
2,tweet3,1,3.0
3,tweet4,0,3.0
4,tweet5,0,3.0


In [12]:
out_df.to_csv('sub0606_3_single.csv',header=True,index=False)